In [6]:
import argparse
import tensorflow as tf
import glovedata
import tensorglove
from glovedata import FEATURES
import tensorglove_osc_server
from sklearn.model_selection import KFold, cross_val_score

SyntaxError: invalid syntax (tensorglove.py, line 60)